In [ ]:
import os
from tqdm import tqdm
import numpy as np

def reading_xdmfs(init=0,final=10000,polydisp=1):
    
    """
    Importe facilmente os arquivos .xdmf da simulação realizada pelo i3d por meio desta função.
    
    O usuário não deve se preocupar com os argumentos a não ser que tenha mais de um tipo de partícula em seus dados, devendo alterar o último argumento.
    
    Todos os dados exportados pelo i3d serão armazenados numa grande biblioteca de dicionários chamada big_picture. 
    
    Para descobrir qual caminho seguir, chamar uma célula com list(big_picture).
    
    """
    
    #starting variables
    
    dtype=0
    
    global big_picture
    
    big_picture={'xy_planes':{}, 'xz_planes':{}, 'yz_planes':{}, '3d_snapshots':{}}
    
    #all possible ways to follow data
    
    for planes in ['xy_planes','xz_planes','yz_planes','3d_snapshots']:

        create_dimension=[]
        
        #all files or only some
        
        for loop in tqdm(range(init,final,1)):
        
            loop_str="{:04d}".format(loop)

            if os.path.exists(f'./data/{planes}_{loop_str}.xdmf')==True:

                with open(f'./data/{planes}_{loop_str}.xdmf', 'r') as f:

                    lines = f.read().splitlines()

                    for i in range(0,len(lines)):

                        #setting type of float
                            
                        if dtype==0: 
                            if 'Precision="4"' in lines[i]:
                                dtype=np.float32
                            elif 'Precision="8"' in lines[i]:
                                dtype=np.float64

                        #setting nx, ny and nz
                                
                        if create_dimension==[]:
                            if 'Dimensions="    ' in lines[i]:
                                for check_dim in lines[i][:]:
                                    if check_dim.isdigit()==True or check_dim.isspace()==True:
                                        create_dimension.append(check_dim)

                                add_index=[]

                                for i in range(0,len(create_dimension),1):
                                    if create_dimension[i].isdigit()==True and create_dimension[i-1].isspace()==True:
                                        add_index.append(i)
                                    if create_dimension[i-1].isdigit()==True and create_dimension[i].isspace()==True:
                                        add_index.append(i)

                                nz=int(''.join(create_dimension[add_index[0]:add_index[1]]))
                                ny=int(''.join(create_dimension[add_index[2]:add_index[3]]))
                                nx=int(''.join(create_dimension[add_index[4]:add_index[5]]))

                        #setting name of files
                                
                        if '</DataItem>' in lines[i]:

                            create_name=[]
                            if planes!='3d_snapshots':
                                
                                #getting only what matters for planes
                                
                                for check_name in lines[i-1][26:]:
                                    if check_name.isalpha()==True:
                                        create_name.append(check_name)
                            
                            elif planes=='3d_snapshots':
                                
                                #getting only what matters for 3d_snapshots
                                
                                for check_name in lines[i-1][29:]: 
                                    if check_name.isalpha()==True:
                                        create_name.append(check_name)

                            create_name=''.join(create_name)
                            
                            #adding polydisperse planes

                            if create_name[:4]=='phib' or create_name[:4]=='phit' or create_name[:4]=='phic' or create_name[:3]=='phi':                               
                                for count in range(1,polydisp+1,1):
                                    create_name=create_name[:3]+f'{count}'+create_name[3:]
                                    
                            #getting rid of trash

                            if create_name!='EE' and create_name!='' and create_name!='ibm': 
                                if loop==init:
                                    big_picture[planes][create_name]={}

                                if planes=='3d_snapshots':
                                    n1,n2,n3=nx,ny,nz
                                elif planes=='xy_planes':
                                    n1,n2=nx,ny
                                elif planes=='xz_planes':
                                    n1,n2=nx,nz
                                elif planes=='yz_planes':
                                    n1,n2=ny,nz
                                    
                                #finally importing files

                                if planes!='3d_snapshots':
                                    big_picture[planes][create_name][loop_str]=[np.fromfile('./data/'+planes+'/'+create_name+loop_str, dtype=dtype).reshape((n1, n2),order='F')]
                                
                                elif planes=='3d_snapshots':
                                    big_picture[planes][create_name][loop_str]=[np.average(np.fromfile('./data/'+planes+'/'+create_name+loop_str, dtype=dtype).reshape((n1, n2, n3), order='F'), axis=-1)]

In [ ]:
reading_xdmfs()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation

fig = plt.figure()

fig.suptitle('Testing reading_xdmfs() function', fontsize=16)

ims = []

for loop in tqdm(range(0,100,1)):
        
    loop_str="{:04d}".format(loop)
    
    im = plt.imshow(big_picture['3d_snapshots']['phi1'][loop_str][0].T,animated=True)
    
    ims.append([im])

ani = animation.ArtistAnimation(fig, ims, interval=250, blit=True,
                                repeat_delay=1000)

ani.save('reading_xdmfs_funcs.mp4')

plt.show()